# Welter issue #9
## Generate synthetic, noised-up two-temperature model spectra, then naively fit a single temperature model to it.
### Part 2- Iterate

Michael Gully-Santiago  
Friday, January 8, 2015  

We now need to iterate.

In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt

In [2]:
% matplotlib inline
% config InlineBackend.figure_format = 'retina'
import seaborn as sns
sns.set_context('notebook')

See the previous notebook for the theory and background.

In [3]:
import os
import json
import pandas as pd
import yaml

## Make mixture model
Assumes you have run splot.py and renamed the model json file to model_a and model_b.

In [4]:
def make_mixture_model():
    try:
        with open('model_A_spec.json') as f:
            ma_raw = json.load(f)

        with open('model_B_spec.json') as f:
            mb_raw = json.load(f)
    except FileNotFoundError:
        print("You need to have the model_A and model_B spectral json files in place first.")
        
    r_factor = 1.0/2.39894
    wl = np.array(ma_raw['wl'])
    fl_ma = np.array(ma_raw['model'])
    nd_ma = np.array(ma_raw['noise_draw'])
    fl_mb_raw = np.array(mb_raw['model'])
    nd_mb_raw = np.array(mb_raw['noise_draw'])

    #Scale the flux (and noise spectrum) by the cofactor $r$ as discussed above.
    fl_mb = fl_mb_raw * r_factor
    nd_mb = nd_mb_raw * r_factor

    # Mixture model
    c = 0.70

    mmix = c * fl_ma + (1 - c) * fl_mb
    ndmix = 0.5 * (c * nd_ma + (1 - c) * nd_mb) # Remove the factor of 2.

    mmix_noised = mmix + ndmix
    return {"wl":wl.tolist(), "mmix":mmix.tolist(), "mmix_noised":mmix_noised.tolist()}

## Automate the mixture model making

In [5]:
os.chdir('/Users/gully/GitHub/welter/notebooks/')

In [ ]:
for i in range(0, 34+1):
    print(i)
    os.chdir('../sf/eo{:03d}/'.format(i))

    # Make model A
    with open('config.yaml', mode='r') as f:
        config = yaml.load(f)    

    config['Theta']['grid'] = [4100.0, 3.5, 0.0]
    with open('config.yaml', mode='w') as f:
        f.write(yaml.dump(config))

    os.system('star.py --generate')
    os.system('splot.py s0_o0spec.json --matplotlib --noise --save_draw')
    os.system('mv model_spec.json model_A_spec.json')

    # Make model B
    with open('config.yaml', mode='r') as f:
        config = yaml.load(f)    

    config['Theta']['grid'] = [3300.0, 3.5, 0.0]
    with open('config.yaml', mode='w') as f:
        f.write(yaml.dump(config))

    os.system('star.py --generate')
    os.system('splot.py s0_o0spec.json --matplotlib --noise --save_draw')
    os.system('mv model_spec.json model_B_spec.json')

    # Make mixture model
    my_dict = make_mixture_model()

    # Save mixture model
    with open("mixture_model.json", mode='w') as f:
        json.dump(my_dict, f, indent=2, sort_keys=True)

    os.chdir('/Users/gully/GitHub/welter/notebooks/')